# L41: Lab 2 - Kernel implications of IPC

The goals of this lab are to:
    
- Continue to gain experience tracing user-kernel interactions via system calls
- Explore the performance of varying IPC models, buffer sizes, and process models
- Gather data to support writing your first assessed lab report

You will do this by using DTrace to analyse the behaviour of a potted, kernel-intensive IPC benchmark.

## IPC benchmark

### Compiling the benchmark

In [ ]:
# Build the benchmark
!make -C ipc

### Running the benchmark

Once built, you can run the benchmark binaries as follows, with command-line arguments specifying various benchmark parameters:

In [ ]:
# Execute the ipc-static benchmark displaying the command line options
!ipc/ipc-static

In [ ]:
# Execute the ipc-dynamic benchmark displaying the command line options
!ipc/ipc-dynamic

### Example benchmark commands

This command performs a simple IPC benchmark using a pipe and default userspace IPC buffer and total IPC sizes within a single thread of a single process:

In [ ]:
# Example benchmark command
print_header("Simple IPC benchmark using a single thread of a single process")

!ipc/ipc-static -i pipe 1thread
    
print_footer("Completed")

How do the six basic modes of the benchmark compare in terms of IPC throughput: {1thread, 2thread, 2proc} × {pipe, socket}?

In [ ]:
# Example benchmark command
print_header("IPC throughput: {1thread, 2thread, 2proc} × {pipe, socket}")

output_pipe_1thread = !ipc/ipc-static -i pipe 1thread
output_pipe_2thread = !ipc/ipc-static -i pipe 2thread
output_pipe_2proc = !ipc/ipc-static -i pipe 2proc
    
output_local_1thread = !ipc/ipc-static -i local 1thread
output_local_2thread = !ipc/ipc-static -i local 2thread
output_local_2proc = !ipc/ipc-static -i local 2proc

# Display comparison of the bandwidth for the benchmark with  {1thread, 2thread, 2proc} × {pipe, socket}
print "Bandwidth -i pipe 1thread = " + output_pipe_1thread[0]
print "Bandwidth -i pipe 2thread = " + output_pipe_2thread[0]
print "Bandwidth -i pipe 2proc = " + output_pipe_2proc[0]
print "Bandwidth -i local 1thread = " + output_local_1thread[0]
print "Bandwidth -i local 2thread = " + output_local_2thread[0]
print "Bandwidth -i local 2proc = " + output_local_2proc[0]

print_footer("Completed")

How do distributions of `read()` and `write()` system-call return values vary from one another within one benchmark?

In [ ]:
# D Language script
io_syscall_script = """
syscall::clock_gettime:return
/execname == "ipc-static" && !self->ipc_loop/
{
    self->ipc_loop = 1;
}

syscall::clock_gettime:entry
/self->ipc_loop/
{
    self->ipc_loop = 0;
}

syscall:::return
/self->ipc_loop/
{
    @a[probefunc] = quantize(arg0);
}
"""

# Callback invoked to process the aggregation
from collections import defaultdict
values = defaultdict(dict)

def simple_out(a, b, c, d):
    outer_key = c[0]
    for kv in d:
        inner_key = kv[0]
        value = kv[1]
        if value != 0:
            values[outer_key][inner_key] = value

# Create a seperate thread to run the DTrace instrumentation
dtrace_thread = DTraceConsumerThread(io_syscall_script,
                                    walk_func=simple_out,
                                    sleep=1)

# Start the DTrace instrumentation
output = dtrace_thread.start()

# Display header to indicate that the benchmarking has started
print_header("Starting ipc-static benchmark using pipe PIC")

!ipc/ipc-static -B -q -i local 1thread
        
# The benchmark has completed - stop the DTrace instrumentation
dtrace_thread.stop()
dtrace_thread.join()
dtrace_thread.consumer.__del__()    

# read() AND WRITE() RETURN CODE DISTRIBUTION
print("read() return code distribution {}".format(values['read']))
print("write() return code distribution {}".format(values['write']))

# Display footer to indicate that the benchmarking has finished
print_footer("Finished ipc-static benchmark")